<a href="https://colab.research.google.com/github/aethelind/notebooks-misc/blob/main/Copy_of_most_simplified_aaai_melding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## submodular.py

In [31]:
import torch
from torch.autograd import Variable
import numpy as np
import scipy as sp
import scipy.sparse
import scipy.linalg


class ContinuousOptimizer(torch.autograd.Function):
    """
    pytorch module for differentiable submodular maximization. The forward pass 
    computes the optimal x for given parameters. The backward pass differentiates 
    that optimal x wrt the parameters.
    """

    @staticmethod
    def forward(ctx, params):
        """
        Computes the optimal x using the supplied optimizer. 
        """
        with torch.enable_grad():
            x = optimize_coverage_multilinear(P=params, verbose=True, k=20, c=0.95, minibatch_size=None)
        ctx.save_for_backward(params, x) 
        return x.data

    @staticmethod
    def backward(ctx, grad_output):
        """
        Differentiates the optimal x returned by the forward pass with respect
        to the ratings matrix that was given as input.
        """
        params, x = ctx.saved_tensors 
        xgrad = x.grad.data
        dxdr = ContinuousOptimizer.get_dxdr(x.data.detach().numpy(), -xgrad.detach().numpy(), params.detach().numpy(), dgrad_coverage, 0.95)
        dxdr_t = torch.from_numpy(np.transpose(dxdr))
        out = torch.mm(dxdr_t.float(), grad_output.view(len(x), 1))
        return out.view_as(params)

    @staticmethod
    def get_dxdr(x, grad, params, get_dgradf_dparams, max_x):
        '''
        Returns the derivative of the optimal solution in the region around x in 
        terms of the rating matrix r. 

        x: an optimal solution

        grad: df/dx at x

        params: the current parameter settings
        '''
        n = len(x)
        # first get the optimal dual variables via the KKT conditions
        # dual variable for constraint sum(x) <= k
        if np.logical_and(x > 0, x < max_x).any():
            lambda_sum = np.mean(grad[np.logical_and(x > 0, x < max_x)])
        else:
            lambda_sum = 0
        # dual variable for constraint x <= max_x
        lambda_upper = []
        # dual variable for constraint x >= 0
        lambda_lower = []
        for i in range(n):
            if np.abs(x[i] - max_x) < 0.000001:
                lambda_upper.append(grad[i] - lambda_sum)
            else:
                lambda_upper.append(0)
            if x[i] > 0:
                lambda_lower.append(0)
            else:
                lambda_lower.append(grad[i] - lambda_sum)
        # number of constraints
        m = 2*n + 1
        # collect value of dual variables
        lam = np.zeros((m))
        lam[0] = lambda_sum
        lam[1:(n+1)] = lambda_upper
        lam[n+1:] = lambda_lower
        diag_lambda = np.matrix(np.diag(lam))
        # collect value of constraints
        g = np.zeros((m))
        # TODO: replace the second x.sum() with k so that this is actually generally correct
        g[0] = x.sum() - x.sum()
        g[1:(n+1)] = x - max_x
        g[n+1:] = -x
        diag_g = np.matrix(np.diag(g))
        # gradient of constraints wrt x
        dgdx = np.zeros((m, n))
        # gradient of constraint sum(x) <= k
        dgdx[0, :] = 1
        # gradient of constraints x <= 1
        for i in range(1, n+1):
            dgdx[i, i-1] = 1
        # gradient of constraints x >= 0 <--> -x <= 0
        for i in range(n+1, m):
            dgdx[i, i-(n+1)] = -1
        dgdx = np.matrix(dgdx)
        # the Hessian matrix -- all zeros for now
        H = np.matrix(np.zeros((n, n)))
        # coefficient matrix for the linear system
        A = np.bmat([[H, np.transpose(dgdx)], [diag_lambda*dgdx, diag_g]])
        # add 0.01*I to improve conditioning
        A = A + 0.01*np.eye(n+m)
        # RHS of the linear system, mostly partial derivative of grad f wrt params
        dgradf_dparams = get_dgradf_dparams(x, params)
        reshaped = np.zeros(
            (dgradf_dparams.shape[0], dgradf_dparams.shape[1]*dgradf_dparams.shape[2]))
        for i in range(n):
            reshaped[i] = dgradf_dparams[i].flatten()
        b = np.bmat([[reshaped], [np.zeros((m, reshaped.shape[1]))]])
        # solution to the system
        derivatives = sp.linalg.solve(A, b)
        if np.isnan(derivatives).any():
            print('report')
            print(np.isnan(A).any())
            print(np.isnan(b).any())
            print(np.isnan(dgdx).any())
            print(np.isnan(diag_lambda).any())
            print(np.isnan(diag_g).any())
            print(np.isnan(dgradf_dparams).any())
        # first n are derivatives of primal variables
        derivatives = derivatives[:n]
        return derivatives


## coverage.py

In [22]:
import torch
import numpy as np
from numba import jit


@jit
def gradient_coverage(x, P):
    n = P.shape[1]
    m = len(x)
    grad = np.zeros(m, dtype=np.float32)
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        for j in range(m):
            grad[j] += P[j, i] * p_all_fail/p_fail[j]
    return grad


@jit
def objective_coverage(x, P):
    n = P.shape[1]
    total = 0
    for i in range(n):
        p_fail = 1 - x*P[:, i]
        p_all_fail = np.prod(p_fail)
        total += (1 - p_all_fail)
    return total


class CoverageInstanceMultilinear(torch.autograd.Function):
    """
    Represents a coverage instance with given coverage probabilities
    P. Forward pass computes the objective value (if evaluate_forward
    is true). Backward computes the gradients w.r.t. decision variables x.
    """
    @staticmethod
    def forward(ctx, x, P):
        ctx.save_for_backward(x, P)
        out = objective_coverage(x.detach().numpy(), P.detach().numpy())
        return torch.tensor(out).float()

    @staticmethod
    def backward(ctx, grad_in):
        x, P = ctx.saved_tensors
        grad = gradient_coverage(x.detach().numpy(), P.detach().numpy())
        return torch.from_numpy(grad).float()*grad_in.float(), None


def optimize_coverage_multilinear(P, verbose=True, k=10, c=1., minibatch_size=None):
    '''
    Run some variant of SGD for the coverage problem with given 
    coverage probabilities P.
    '''
    # decision variables
    x = torch.zeros(P.shape[0], requires_grad=True)
    # set up the optimizer
    optimizer = torch.optim.SGD([x], momentum=0.9, lr=0.1, nesterov=True)
    # take projected stochastic gradient steps
    for t in range(20):
        # objective which will provide gradient evaluations
        loss = -CoverageInstanceMultilinear.apply(x, P)
        #if verbose:
            #print(t, -loss.item())
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        x.data = torch.from_numpy(project_uniform_matroid_boundary(x.data.numpy(), k, 1/c)).float()
    return x


@jit
def dgrad_coverage(x, P):
    n = P.shape[1]
    m = len(x)
    dgrad = np.zeros((m, m, n), dtype=np.float32)
    for i in range(n):
        p_fail = 1 - x*P[:,i]
        p_all_fail = np.prod(p_fail)
        for j in range(m):
            for k in range(m):
                if j == k:
                    dgrad[j, k, i] = p_all_fail/p_fail[j]
                else:
                    dgrad[j, k, i] = -x[k] * P[j, i] * p_all_fail/(p_fail[j] * p_fail[k])
    return dgrad

## utils.py

In [10]:
def project_uniform_matroid_boundary(x, k, c=1):
    '''
    Exact projection algorithm of Karimi et al. This is the projection implementation
    that should be used now.
    
    Projects x onto the set {y: 0 <= y <= 1/c, ||y||_1 = k}
    '''
    import numpy as np
    k *= c
    n = len(x)
    x = x.copy()
    alpha_upper = x/c
    alpha_lower = (x*c - 1)/c**2
    S = []
    S.extend(alpha_lower)
    S.extend(alpha_upper)
    S.sort()
    S = np.unique(S)
    h = n
    alpha = min(S) - 1
    m = 0
    for i in range(len(S)):
        hprime = h + (S[i] - alpha)*m
        if hprime < k and k <= h:
            alphastar = (S[i] - alpha)*(h - k)/(h - hprime) + alpha
            result = np.zeros((n))
            for j in range(n):
                if alpha_lower[j] > alphastar:
                    result[j] = 1./c
                elif alpha_upper[j] >= alphastar:
                    result[j] = x[j] - alphastar*c
            return result
        m -= (alpha_lower == S[i]).sum()*(c**2)
        m += (alpha_upper == S[i]).sum()*(c**2)
        h = hprime
        alpha = S[i]
    raise Exception('projection did not terminate')

In [1]:
# Clear out directory
!rm -rf *
# Download data_decisions_benchmarks.zip and unzip diverse_recommendation_data.pickle
!curl https://bryanwilder.github.io/files/data_decisions_benchmarks.zip | jar xv benchmarks_release/diverse_recommendation_data.pickle
# Move diverse_recommendation_data.pickle to current directory
!mv benchmarks_release/diverse_recommendation_data.pickle .
# Remove empty directory
!rm -rf benchmarks_release
# Download hetrec2011-movielens-2k-v2.zip and unzip movie_actors.dat and user_ratedmovies.dat
!curl https://files.grouplens.org/datasets/hetrec2011/hetrec2011-movielens-2k-v2.zip | jar xv movie_actors.dat user_ratedmovies.dat

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 83.0M  100 83.0M    0     0  22.6M      0  0:00:03  0:00:03 --:--:-- 22.6M
 inflated: benchmarks_release/diverse_recommendation_data.pickle
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 14 17.9M   14 2736k    0     0  6772k      0  0:00:02 --:--:--  0:00:02 6755k inflated: movie_actors.dat
 inflated: user_ratedmovies.dat
100 17.9M  100 17.9M    0     0  18.1M      0 --:--:-- --:--:-- --:--:-- 18.1M


In [2]:
## recommendation_nn_decision.py
## movie problem
import numpy as np
import torch
import pickle
from functools import partial
import torch.nn as nn
import random

num_layers = 1
activation = 'relu'
#k = 20
#use_hessian = False
num_iters = 100
instance_sizes = [100]
learning_rate = 1e-4

Ps = {}
data = {}
f_true = {}
for num_items in instance_sizes:
    with open('diverse_recommendation_data' + '.pickle', 'rb') as f:
        # Ps_size takes on the actors 100x500 and data_size takes the users 100x2113
        Ps_size, data_size = pickle.load(f)

    num_targets = Ps_size[0].shape[1]
    num_features = data_size[0].shape[1]
    Ps[num_items] = [torch.from_numpy(P).long() for P in Ps_size]
    data[num_items] = [torch.from_numpy(x).float() for x in data_size]
    w = np.ones(num_targets, dtype=np.float32)
    f_true[num_items] = [(P, w) for P in Ps[num_items]]


num_repetitions = 0

train = {}
test = {}
for size in instance_sizes:
    with open('diverse_recommendation_data' + '.pickle', 'rb') as f:
        train[size], test[size] = pickle.load(f)


In [7]:
Ps_size[0].shape

(100, 500)

In [11]:
Ps_size[0][0].sum()

4.0

In [8]:
data_size[0].shape

(100, 2113)

# Train

In [35]:
vals = np.zeros((num_repetitions+30, len(instance_sizes), len(instance_sizes)))

for idx in range(num_repetitions, num_repetitions+1): #+ 30):

    intermediate_size = 200
    def make_fc():
        if num_layers > 1:
            if activation == 'relu':
                activation_fn = nn.ReLU
            elif activation == 'sigmoid':
                activation_fn = nn.Sigmoid
            else:
                raise Exception(
                    'Invalid activation function: ' + str(activation))
            net_layers = [
                nn.Linear(num_features, intermediate_size), activation_fn()]
            for hidden in range(num_layers-2):
                net_layers.append(
                    nn.Linear(intermediate_size, intermediate_size))
                net_layers.append(activation_fn())
            net_layers.append(nn.Linear(intermediate_size, num_targets))
            net_layers.append(nn.Sigmoid())
            return nn.Sequential(*net_layers)
        else:
            return nn.Sequential(nn.Linear(num_features, num_targets), nn.Sigmoid())

    # runs the given net on instances of a given size
    def eval_opt(net, instances, size):
        net.eval()
        val = 0.
        for i in range(len(instances)):
            pred = net(data[size][i])
            x = ContinuousOptimizer.apply(pred)
            pp, _ = f_true[size][i] #audrey fix
            val += objective_coverage(x.detach().numpy().round(), pp.detach().numpy()) #audrey fix
        net.train()
        return val/len(instances)

    # train a network for each size, and test on each sizes
    for train_idx, train_size in enumerate(instance_sizes):
        net = make_fc()
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
        # training
        for t in range(num_iters):
            print(f"Iteration {t}")
            ### print("Get the model predictions...") ###
            i = random.randint(0, 80)
            y = data[train_size][i]
            pred = net(y)
            ### print("Get the optimal solution to the continous problem...")
            x = ContinuousOptimizer.apply(pred)
            ### print("Get the objective value and set as the loss...")
            pp, _ = f_true[train_size][i]
            loss = -CoverageInstanceMultilinear.apply(x, pp)
            print("training loss", loss)
            ### print("Update model weights based on the computed gradients...")
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            #break
        #break
        # save learned network state
        savepath = '/tmp/net_diffopt_smalllr_{0}_{1}_{2}_{3}.pt'.format(  #audrey fixes /tmp/
            train_size, 2, num_layers, idx)
        torch.save(net.state_dict(), savepath)
        # test on different sizes
        for test_idx, test_size in enumerate(instance_sizes):
            vals[idx, train_idx, test_idx] = eval_opt(
                net, test, test_size)
            print(vals[idx, train_idx, test_idx])
        # save out values
        print(idx, train_size, vals[idx, train_idx])
        with open('results_recommendation_' + str(num_layers) + '.pickle', 'wb') as f:
            pickle.dump(vals, f)
    #break


Iteration 0
training loss tensor(-17.0664, grad_fn=<NegBackward0>)
Iteration 1
training loss tensor(-45.8804, grad_fn=<NegBackward0>)
Iteration 2
training loss tensor(-51.6705, grad_fn=<NegBackward0>)
Iteration 3
training loss tensor(-60.3399, grad_fn=<NegBackward0>)
Iteration 4
training loss tensor(-46.5997, grad_fn=<NegBackward0>)
Iteration 5
training loss tensor(-23.5520, grad_fn=<NegBackward0>)
Iteration 6
training loss tensor(-53.4883, grad_fn=<NegBackward0>)
Iteration 7
training loss tensor(-29.5868, grad_fn=<NegBackward0>)
Iteration 8
training loss tensor(-48.5714, grad_fn=<NegBackward0>)
Iteration 9
training loss tensor(-54.6399, grad_fn=<NegBackward0>)
Iteration 10
training loss tensor(-43.0479, grad_fn=<NegBackward0>)
Iteration 11
training loss tensor(-29.0924, grad_fn=<NegBackward0>)
Iteration 12
training loss tensor(-51.0705, grad_fn=<NegBackward0>)
Iteration 13
training loss tensor(-56.6278, grad_fn=<NegBackward0>)
Iteration 14
training loss tensor(-69.7474, grad_fn=<NegBa

In [33]:
x.round()

tensor([1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
        0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], grad_fn=<RoundBackward0>)

In [34]:
pred

tensor([[0.6899, 0.9460, 0.9289,  ..., 0.7512, 0.7547, 0.5881],
        [0.7172, 0.3402, 0.3255,  ..., 0.5605, 0.7332, 0.8300],
        [0.6713, 0.4358, 0.6057,  ..., 0.6982, 0.6583, 0.3758],
        ...,
        [0.5496, 0.4726, 0.4089,  ..., 0.5988, 0.4816, 0.5596],
        [0.5360, 0.4260, 0.4028,  ..., 0.4339, 0.4665, 0.4471],
        [0.5491, 0.4958, 0.4940,  ..., 0.5498, 0.4901, 0.4619]],
       grad_fn=<SigmoidBackward0>)

In [18]:
## recommendation_nn_decision.py
## synthetic problem
import numpy as np
import torch
import pickle
from functools import partial
import torch.nn as nn
import random

# load probability matrix 
P_list = [
[	0	,	1	,	0	,	1	],
[	0	,	1	,	0	,	0	],
[	1	,	1	,	0	,	1	],
[	0	,	1	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	1	,	0	,	1	],
[	0	,	1	,	0	,	1	],
[	0	,	0	,	0	,	0	],
[	0	,	1	,	0	,	0	],
[	1	,	1	,	0	,	1	],
[	0	,	0	,	1	,	0	],
[	0	,	1	,	0	,	1	],
[	0	,	1	,	0	,	0	],
[	0	,	1	,	0	,	1	],
[	0	,	1	,	0	,	1	],
[	0	,	1	,	0	,	0	],
[	0	,	1	,	0	,	0	],
[	0	,	0	,	0	,	0	],
[	0	,	0	,	1	,	0	],
[	0	,	1	,	0	,	1	],
[	0	,	1	,	0	,	1	],
[	1	,	1	,	0	,	1	],
[	0	,	0	,	1	,	0	],
[	0	,	1	,	0	,	1	],
]

# load features
circuit_km = [
3.7, 
3, 
6.9, 
3.5, 
2.2, 
2.8, 
2.4, 
4.5, 
4.3, 
2.6, 
3.2, 
5.4, 
1.2, 
4.1, 
3.4, 
4.3, 
4, 
3.3, 
3.4, 
2.4, 
1.2, 
4.5, 
4, 
4.9, 
0.25, 
3.9, 
]
y = []
for i in circuit_km:
  y.append([i])

num_layers = 1
activation = 'relu'
#k = 2
#use_hessian = False
num_iters = 50
instance_sizes = [0]
learning_rate = 1e-4

Ps = {}
data = {}
f_true = {}

for num_items in instance_sizes:
    Ps_size = np.array(P_list)
    data_size = np.array(y)

    num_targets = Ps_size.shape[1] #500 --> 4
    num_features = data_size.shape[1] #2113 --> 1
    Ps[num_items] = [torch.from_numpy(Ps_size).long()]
    data[num_items] = [torch.from_numpy(data_size).float()]
    w = np.ones(num_targets, dtype=np.float32)
    f_true[num_items] = [(P, w) for P in Ps[num_items]]
  
num_repetitions = 0

train = {}
test = {}
for size in instance_sizes:
  train[size], test[size] = np.array(P_list), np.array(y)

## Run Optimizer on Ground Truth

In [ ]:
x_ground = ContinuousOptimizer.apply()